In [ ]:
import pandas as pd
df=pd.read_csv("sample_data/train_metrics.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27419 entries, 0 to 27418
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   repo               27419 non-null  object 
 1   file_path          27419 non-null  object 
 2   raw_loc            27419 non-null  int64  
 3   raw_lloc           27419 non-null  int64  
 4   raw_sloc           27419 non-null  int64  
 5   raw_comments       27419 non-null  int64  
 6   raw_multi          27419 non-null  int64  
 7   raw_blank          27419 non-null  int64  
 8   h1                 27419 non-null  int64  
 9   h2                 27419 non-null  int64  
 10  N1                 27419 non-null  int64  
 11  N2                 27419 non-null  int64  
 12  vocabulary         27419 non-null  int64  
 13  length             27419 non-null  int64  
 14  calculated_length  27419 non-null  float64
 15  volume             27419 non-null  float64
 16  difficulty         274

In [ ]:
data1=[v for k, v in df.groupby('repo')]

In [ ]:
import numpy as np
dataf=[]
for i in range(0,len(data1)):
    data2=data1[i]
    data3=data2.iloc[:,2:]
    data3=np.array(data3)
    if np.sum(data3[:,-1])/data1[i].shape[0]*100>=10 and np.sum(data3[:,-1])/data1[i].shape[0]*100<=70:
        dataf.append(data3)
        print(i,'sel',data2.iloc[0,0])
    print(i,data1[i].shape,np.sum(data3[:,-1]))

0 (1940, 22) 1924.0
1 (757, 22) 744.0
2 sel black
2 (149, 22) 71.0
3 sel celery
3 (2297, 22) 681.0
4 (2515, 22) 2461.0
5 sel cpython
5 (1992, 22) 1233.0
6 (3295, 22) 3190.0
7 sel django-rest-framework
7 (387, 22) 264.0
8 sel jax
8 (396, 22) 241.0
9 (10, 22) 9.0
10 sel localstack
10 (520, 22) 242.0
11 sel numpy
11 (490, 22) 244.0
12 sel openpilot
12 (811, 22) 404.0
13 (2622, 22) 2553.0
14 sel pipenv
14 (22, 22) 8.0
15 sel poetry
15 (265, 22) 116.0
16 (1215, 22) 1189.0
17 sel redash
17 (485, 22) 139.0
18 (1672, 22) 1230.0
19 sel scrapy
19 (245, 22) 134.0
20 (2400, 22) 2350.0
21 sel spaCy
21 (1690, 22) 937.0
22 sel transformers
22 (594, 22) 228.0
23 sel yolov5
23 (650, 22) 422.0


In [ ]:
len(dataf)

15

In [ ]:
for i in range(0,15):
  print(np.shape(dataf[i]),np.sum(dataf[i][:,-1]))

(149, 20) 71.0
(2297, 20) 681.0
(1992, 20) 1233.0
(387, 20) 264.0
(396, 20) 241.0
(520, 20) 242.0
(490, 20) 244.0
(811, 20) 404.0
(22, 20) 8.0
(265, 20) 116.0
(485, 20) 139.0
(245, 20) 134.0
(1690, 20) 937.0
(594, 20) 228.0
(650, 20) 422.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
floc='/content/drive/MyDrive/pyfault/'

In [ ]:
c=0
for i in range(0,15):
  if np.shape(dataf[i])[0]>=50:
    fname=floc+'data'+str(c)+'.csv'
    np.savetxt(fname,dataf[i],delimiter=',',fmt='%f')
    c=c+1

In [ ]:
for i in range(0,14):
  fname=floc+'data'+str(i)+'.csv'
  data=np.genfromtxt(fname,delimiter=',')
  print(np.shape(data))

(149, 20)
(2297, 20)
(1992, 20)
(387, 20)
(396, 20)
(520, 20)
(490, 20)
(811, 20)
(265, 20)
(485, 20)
(245, 20)
(1690, 20)
(594, 20)
(650, 20)


In [ ]:
import math
import pandas as pd
import numpy as np
from matplotlib import pyplot

In [ ]:
from scipy.stats import ranksums
for i in range(0,14):
  fname=floc+'data'+str(i)+'.csv'
  data=np.genfromtxt(fname,delimiter=',')
  p=np.zeros((np.shape(data)[1]))
  in1=np.where(data[:,-1]==0)
  in2=np.where(data[:,-1]==1)
  for j in range(0,np.shape(data)[1]-1):
    p1=ranksums(data[in1[0],j],data[in2[0],j])
    p[j]=p1.pvalue
  in1=np.where(p<=0.05)
  if len(in1[0])==1:
    in1=np.where(p<=0.5)
    if len(in1[0])==1:
      in1=np.where(p<=0.8)
  datan=data[:,in1[0]]
  fname=floc+'data'+str(14+i)+'.csv'
  np.savetxt(fname,datan,delimiter=',',fmt='%f')
  print(i,len(in1[0]))

0 4
1 20
2 3
3 20
4 20
5 6
6 6
7 13
8 20
9 5
10 7
11 20
12 19
13 8


In [ ]:
def cosscorr(data):
  r=np.corrcoef(data[:,0:-1],rowvar=False)
  r=np.abs(r)
  imp=np.ones((np.shape(data)[1]))
  for i in range(0,np.shape(data)[1]-1):
    if imp[i]==1:
      in1=np.where(r[i,i+1:]>=0.7)
      imp[i+1+in1[0]]=0
  return imp

In [ ]:
for i in range(0,14):
  fname=floc+'data'+str(i)+'.csv'
  data=np.genfromtxt(fname,delimiter=',')
  imp=cosscorr(data)
  in1=np.where(imp==1)
  datan=data[:,in1[0]]
  print(i,len(in1[0]))
  datan=np.concatenate((datan,data[:,-1].reshape(-1,1)), axis=1)
  fname=floc+'data'+str(28+i)+'.csv'
  np.savetxt(fname,datan,delimiter=',', fmt='%f')

0 3
1 3
2 5
3 2
4 3
5 2
6 4
7 5
8 3
9 4
10 5
11 3
12 6
13 4


In [ ]:
from sklearn.feature_selection import chi2
for i in range(0,14):
  fname=floc+'data'+str(i)+'.csv'
  data=np.genfromtxt(fname,delimiter=',')
  chi2_stats, p_values = chi2(abs(data[:,0:-1]),data[:,-1])
  in1=np.where(p_values<=0.05)
  datan=data[:,in1[0]]
  datan=np.concatenate((datan,data[:,-1].reshape(-1,1)), axis=1)
  fname=floc+'data'+str(42+i)+'.csv'
  np.savetxt(fname,datan,delimiter=',',fmt='%f')
  print(i,len(in1[0]))

0 17
1 19
2 15
3 16
4 19
5 14
6 16
7 18
8 18
9 16
10 9
11 17
12 19
13 11


In [ ]:
from sklearn.decomposition import PCA
import math
for i in range(0,14):
  print(i)
  fname=floc+'data'+str(i)+'.csv'
  data=np.genfromtxt(fname,delimiter=',')
  n=int(math.log2(np.shape(data)[1]-1))
  pca = PCA(n_components=n)
  pca.fit(data[:,0:-1])
  datan=pca.transform(data[:,0:-1])
  datan=np.concatenate((datan,data[:,-1].reshape(-1,1)), axis=1)
  fname=floc+'data'+str(56+i)+'.csv'
  np.savetxt(fname,datan,delimiter=',', fmt='%f')

0
1
2
3
4
5
6
7
8
9
10
11
12
13


In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
kf = KFold(5)
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
def normalizedata(X_train):
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    return X_train
from sklearn.metrics import accuracy_score
def accm(data,clss):
  model=DecisionTreeClassifier()
  kf = KFold(5)
  pred=np.zeros((np.shape(data)[0],2))
  for train_index, test_index in kf.split(data):
    trdata=data[train_index,:]
    tsdata=data[test_index,:]
    tract=clss[train_index]
    tsact=clss[test_index]
    model.fit(trdata,tract)
    pred[test_index,0]=model.predict(tsdata)
    pred[test_index,1]=tsact
  return accuracy_score(pred[:,1],pred[:,0])

In [ ]:
import random
def fitness(data,chr):
  fit=np.zeros((50,3))
  for i in range(0,50):
    c=chr[i,:]
    in1=np.where(c==1)
    acc=accm(data[:,in1[0]],data[:,-1])
    fit[i,0]=acc
    fit[i,1]=len(in1[0])
    fit[i,2]=0.8*acc+0.2*(np.shape(data)[1]-len(in1[0]))/np.shape(data)[1]
  return fit
def crossover(chr,noc):
  an=random.sample(range(0, noc),noc)
  nchr=np.zeros((int(noc/2),np.shape(data)[1]))
  mid=int(np.shape(data)[1]/2)
  for i in range(0,int(noc/2)):
    nchr[i,0:mid]=chr[an[2*i],0:mid]
    nchr[i,mid:]=chr[an[2*i+1],mid:]
  return nchr

In [ ]:
def mutation(chr):
  pos=np.random.randint(0,np.shape(data)[1],[25,5])
  for i in range(0,25):
    chr[i,pos[i,0]]=1-chr[i,pos[i,0]]
    chr[i,pos[i,1]]=1-chr[i,pos[i,1]]
    chr[i,pos[i,2]]=1-chr[i,pos[i,2]]
    chr[i,pos[i,3]]=1-chr[i,pos[i,3]]
    chr[i,pos[i,4]]=1-chr[i,pos[i,4]]
  return chr

In [ ]:
def gafs(data):
  chr=np.random.randint(0,2,[50,np.shape(data)[1]])
  gen=0
  bfit=0
  bchr=0
  while gen<=30:
    fit=fitness(data,chr)
    cfit=np.max(fit[:,2])
    cpos=np.argmax(fit[:,2])
    if cfit>bfit:
      bfit=cfit
      bchr=chr[cpos,:]
      nof=fit[cpos,1]
      bacc=fit[cpos,0]
    chr[0:25,:]=crossover(chr,50)
    chr[0:25,:]=mutation(chr[0:25,:])
    chr[25:,:]=np.random.randint(0,2,[25,np.shape(data)[1]])
    gen=gen+1
    print(bfit,'----->',nof,'----->',bacc)
  return bchr

In [ ]:
for i in range(0,14):
  print(i)
  fname=floc+'data'+str(i)+'.csv'
  data=np.genfromtxt(fname,delimiter=',')
  print('ACC with all features=',accm(data[:,0:-1],data[:,-1]))
  in1=gafs(data)
  in1=np.where(in1==1)
  datan=data[:,in1[0]]
  datan=np.concatenate((datan,data[:,-1].reshape(-1,1)), axis=1)
  fname=floc+'data'+str(70+i)+'.csv'
  np.savetxt(fname,datan,delimiter=',', fmt='%f')

0
ACC with all features= 0.4966442953020134
0.9400000000000001 -----> 6.0 -----> 1.0
0.9500000000000001 -----> 5.0 -----> 1.0
0.9500000000000001 -----> 5.0 -----> 1.0
0.9500000000000001 -----> 5.0 -----> 1.0
0.9500000000000001 -----> 5.0 -----> 1.0
0.9500000000000001 -----> 5.0 -----> 1.0
0.9500000000000001 -----> 5.0 -----> 1.0
0.9500000000000001 -----> 5.0 -----> 1.0
0.9500000000000001 -----> 5.0 -----> 1.0
0.9500000000000001 -----> 5.0 -----> 1.0
0.9600000000000001 -----> 4.0 -----> 1.0
0.9600000000000001 -----> 4.0 -----> 1.0
0.9600000000000001 -----> 4.0 -----> 1.0
0.9600000000000001 -----> 4.0 -----> 1.0
0.9600000000000001 -----> 4.0 -----> 1.0
0.9600000000000001 -----> 4.0 -----> 1.0
0.9600000000000001 -----> 4.0 -----> 1.0
0.9600000000000001 -----> 4.0 -----> 1.0
0.9600000000000001 -----> 4.0 -----> 1.0
0.9600000000000001 -----> 4.0 -----> 1.0
0.9600000000000001 -----> 4.0 -----> 1.0
0.9600000000000001 -----> 4.0 -----> 1.0
0.9600000000000001 -----> 4.0 -----> 1.0
0.96000000000

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn import svm

In [ ]:
def modeln1(trdata,tract,tsdata):
    model = MultinomialNB()
    model.fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1[:,1]
def modeln2(trdata,tract,tsdata):
    model = BernoulliNB()
    model.fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1[:,1]
def modeln3(trdata,tract,tsdata):
    model = GaussianNB()
    model.fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1[:,1]
def modeln4(trdata,tract,tsdata):
    model = DecisionTreeClassifier().fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1[:,1]
def modeln41(trdata,tract,tsdata):
    model = LogisticRegression().fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1[:,1]
def modeln42(trdata,tract,tsdata):
    model = KNeighborsClassifier().fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1[:,1]

In [ ]:
def modeln5(trdata,tract,tsdata):
    model = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(trdata.shape[1], 2), random_state=1,max_iter=1500).fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1[:,1]
def modeln6(trdata,tract,tsdata):
    model = MLPClassifier(solver='sgd', hidden_layer_sizes=(trdata.shape[1], 2), random_state=1,max_iter=1500).fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1[:,1]
def modeln7(trdata,tract,tsdata):
    model = MLPClassifier(solver='adam', hidden_layer_sizes=(trdata.shape[1], 2), random_state=1,max_iter=1500).fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1[:,1]

In [ ]:
def normalizedata(X_train):
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    return X_train
def modeln8(trdata,tract,tsdata):
    model = BaggingClassifier(KNeighborsClassifier(),max_samples=0.5, max_features=0.5).fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1[:,1]
def modeln81(trdata,tract,tsdata):
    model = BaggingClassifier(MultinomialNB(),max_samples=0.5, max_features=0.5).fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1[:,1]
def modeln82(trdata,tract,tsdata):
    model = BaggingClassifier(LogisticRegression(),max_samples=0.5, max_features=0.5).fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1[:,1]
def modeln83(trdata,tract,tsdata):
    model = BaggingClassifier(DecisionTreeClassifier(),max_samples=0.5, max_features=0.5).fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1[:,1]
def modeln9(trdata,tract,tsdata):
    model = RandomForestClassifier(n_estimators=10).fit(trdata,tract)
    model.fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1[:,1]
def modeln10(trdata,tract,tsdata):
    model = ExtraTreesClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0).fit(trdata,tract)
    model.fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1[:,1]
def modeln11(trdata,tract,tsdata):
    model = AdaBoostClassifier(n_estimators=10).fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1[:,1]
def modeln12(trdata,tract,tsdata):
    model = GradientBoostingClassifier(n_estimators=10, learning_rate=1.0,max_depth=1, random_state=0).fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1[:,1]

In [ ]:
def modeln13(trdata,tract,tsdata):
    in0=np.where(tract==0)
    in1=np.where(tract==1)
    wclf = svm.SVC(kernel="linear",probability=True, class_weight={1:int(len(in0[0])/len(in1[0]))})
    wclf.fit(trdata,tract)
    pred= wclf.predict(tsdata)
    pred1=wclf.predict_proba(tsdata)
    return pred,pred1[:,1]

def modeln14(trdata,tract,tsdata):
    in0=np.where(tract==0)
    in1=np.where(tract==1)
    wclf = svm.SVC(kernel="poly",probability=True, class_weight={1:int(len(in0[0])/len(in1[0]))})
    wclf.fit(trdata,tract)
    pred= wclf.predict(tsdata)
    pred1=wclf.predict_proba(tsdata)
    return pred,pred1[:,1]

def modeln15(trdata,tract,tsdata):
    in0=np.where(tract==0)
    in1=np.where(tract==1)
    wclf = svm.SVC(kernel="rbf",probability=True, class_weight={1:int(len(in0[0])/len(in1[0]))})
    wclf.fit(trdata,tract)
    pred= wclf.predict(tsdata)
    pred1=wclf.predict_proba(tsdata)
    return pred,pred1[:,1]

In [ ]:
def modeln131(trdata,tract,tsdata):
    wclf = svm.SVC(kernel="linear",probability=True)
    wclf.fit(trdata,tract)
    pred= wclf.predict(tsdata)
    pred1=wclf.predict_proba(tsdata)
    return pred,pred1[:,1]

def modeln141(trdata,tract,tsdata):
    wclf = svm.SVC(kernel="poly",probability=True)
    wclf.fit(trdata,tract)
    pred= wclf.predict(tsdata)
    pred1=wclf.predict_proba(tsdata)
    return pred,pred1[:,1]

def modeln151(trdata,tract,tsdata):
    wclf = svm.SVC(kernel="rbf",probability=True)
    wclf.fit(trdata,tract)
    pred= wclf.predict(tsdata)
    pred1=wclf.predict_proba(tsdata)
    return pred,pred1[:,1]

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
kf = KFold(3)
for i in range(0,84):
    print(i)
    fname=floc+'data'+str(i)+'.csv'
    data=np.genfromtxt(fname,delimiter=',')
    np.random.shuffle(data)
    predvalue=np.zeros((np.shape(data)[0],16))
    predvalue1=np.zeros((np.shape(data)[0],16))
    data[:,0:-1]=normalizedata(data[:,0:-1])
    for train_index, test_index in kf.split(data):
        trdata=data[train_index,0:-1]
        tsdata=data[test_index,0:-1]
        tract=data[train_index,-1]
        tsact=data[test_index,-1]
        predvalue[test_index,0],predvalue1[test_index,0]=modeln1(trdata,tract,tsdata)
        predvalue[test_index,1],predvalue1[test_index,1]=modeln2(trdata,tract,tsdata)
        predvalue[test_index,2],predvalue1[test_index,2]=modeln3(trdata,tract,tsdata)
        predvalue[test_index,3],predvalue1[test_index,3]=modeln4(trdata,tract,tsdata)
        predvalue[test_index,4],predvalue1[test_index,4]=modeln41(trdata,tract,tsdata)
        predvalue[test_index,5],predvalue1[test_index,5]=modeln42(trdata,tract,tsdata)
        predvalue[test_index,6],predvalue1[test_index,6]=modeln131(trdata,tract,tsdata)
        predvalue[test_index,7],predvalue1[test_index,7]=modeln141(trdata,tract,tsdata)
        predvalue[test_index,8],predvalue1[test_index,8]=modeln151(trdata,tract,tsdata)
        predvalue[test_index,9],predvalue1[test_index,9]=modeln8(trdata,tract,tsdata)
        predvalue[test_index,10],predvalue1[test_index,10]=modeln81(trdata,tract,tsdata)
        predvalue[test_index,11],predvalue1[test_index,11]=modeln82(trdata,tract,tsdata)
        predvalue[test_index,12],predvalue1[test_index,12]=modeln83(trdata,tract,tsdata)
        predvalue[test_index,13],predvalue1[test_index,13]=modeln9(trdata,tract,tsdata)
        predvalue[test_index,14],predvalue1[test_index,14]=modeln10(trdata,tract,tsdata)
        predvalue1[test_index,15]=tsact
        predvalue[test_index,15]=tsact
    fname=floc+str(i)+"pred.csv"
    np.savetxt(fname,predvalue, delimiter=',', fmt='%f')
    fname=floc+str(i)+"predp.csv"
    np.savetxt(fname,predvalue1, delimiter=',', fmt='%f')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83


In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
def roc_curve_and_score(y_test, pred_proba):
    fpr, tpr, _ = roc_curve(y_test.ravel(), pred_proba.ravel())
    roc_auc = roc_auc_score(y_test.ravel(), pred_proba.ravel())
    return fpr, tpr, roc_auc

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
def gran(y1,i,j):
    plt.figure(figsize=(8, 6))
    matplotlib.rcParams.update({'font.size': 14})
    plt.grid()
    fpr, tpr, roc_auc = roc_curve_and_score(y1[:,1],y1[:,0])
    plt.plot(fpr, tpr, color='darkorange', lw=2,
         label='ALLF={0:.3f}'.format(roc_auc))
    fpr, tpr, roc_auc = roc_curve_and_score(y1[:,3],y1[:,2])
    plt.plot(fpr, tpr, color='green', lw=2,
         label='RANKF={0:.3f}'.format(roc_auc))
    fpr, tpr, roc_auc = roc_curve_and_score(y1[:,5],y1[:,4])
    plt.plot(fpr, tpr, color='Orange', lw=2,
         label='CCRAF={0:.3f}'.format(roc_auc))

    fpr, tpr, roc_auc = roc_curve_and_score(y1[:,7],y1[:,6])
    plt.plot(fpr, tpr, color='crimson', lw=2,
         label='CHI2={0:.3f}'.format(roc_auc))

    fpr, tpr, roc_auc = roc_curve_and_score(y1[:,9],y1[:,8])
    plt.plot(fpr, tpr, color='Aqua', lw=2,
         label='PCA={0:.3f}'.format(roc_auc))

    fpr, tpr, roc_auc = roc_curve_and_score(y1[:,11],y1[:,10])
    plt.plot(fpr, tpr, color='Aqua', lw=2,
         label='GA={0:.3f}'.format(roc_auc))

    plt.plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--')
    plt.legend(loc="lower right")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('1 - Specificity')
    plt.ylabel('Sensitivity')

    fna=floc+'tec_'+str(i)+'proj'+'_'+str(j)+".jpg"
    plt.savefig(fna)
    plt.close()

In [ ]:
for l in range(0,14):
  print(l)
  for k in range(0,14):
    for i in range(0,6):
      fname= floc+str(14*i+k)+"predp.csv"
      data=np.genfromtxt(fname,delimiter=',')
      #print(fname)
      if i==0:
        d=np.zeros((np.shape(data)[0],12))
      d[:,2*i]=data[:,l]
      d[:,2*i+1]=data[:,-1]
    gran(d,l,k)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
rocv=np.zeros((84,13))
for i in range(0,84):
    fname= floc+str(i)+"predp.csv"
    data=np.genfromtxt(fname,delimiter=',')
    y1=data[:,-1]
    for j in range(0,13):
        b=data[:,j]
        fpr, tpr, _ = roc_curve(y1, b)
        rocv[i,j]=auc(fpr, tpr)
fname=floc+'auc.csv'
np.savetxt(fname,rocv, delimiter=',', fmt='%f')

In [ ]:
import numpy as np
acv=np.zeros((84,13))
fmea=np.zeros((84,13))
pre=np.zeros((84,13))
rec=np.zeros((84,13))
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import (
    f1_score, precision_score, recall_score,accuracy_score
    )
for i in range(0,84):
    fname= floc+str(i)+"pred.csv"
    data1=np.genfromtxt(fname,delimiter=',')
    y1=data1[:,-1]
    for j in range(0,13):
        acv[i,j]=accuracy_score(y1, data1[:,j])
        fmea[i,j]= f1_score(y1, data1[:,j])
        pre[i,j]= matthews_corrcoef(y1, data1[:,j])

fname=floc+'acc.csv'
np.savetxt(fname,acv, delimiter=',', fmt='%f')
fname=floc+'fmea.csv'
np.savetxt(fname,fmea, delimiter=',', fmt='%f')
fname=floc+'pre.csv'
np.savetxt(fname,pre, delimiter=',', fmt='%f')

In [ ]:
acv

In [ ]:
rocv